<a href="https://colab.research.google.com/github/akaraci/LargeDatasetCovid19/blob/main/Binary_Pre_Trained_and_Modified_Pre_Trained_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autotime
import tensorflow as tf 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D,GlobalAveragePooling2D
import pickle
from keras.models import model_from_json
from keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import scikitplot as skplt
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from tensorflow.keras.optimizers import RMSprop, Adam, Adamax
#from sklearn.metrics import multilabel_confusion_matrix
from sklearn import metrics
from google.colab import drive
drive.mount('/content/drive')
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np
from keras.callbacks import EarlyStopping
np.random.seed(1000)
from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Flatten
import matplotlib.pyplot as plt
import cv2
import numpy as np
from keras.datasets import cifar10
from keras.preprocessing.image import  ImageDataGenerator, img_to_array, load_img
from glob import glob
from keras.models import Model

In [ ]:
##------------Modified and unmodified Pre-trained Architectures's (VGG16, VGG19, InceptionV3 and Resnet50) Binary Class Code ---------------- 
X=pickle.load(open("drive/My Drive/LargeDataSetBen/X_databen_two_az.pickle", "rb"))
y=pickle.load(open("drive/My Drive/LargeDataSetBen/Y_databen_two_az.pickle", "rb"))

# normalizing data (a pixel goes from 0 to 255)
X = X/255.0   
#X = X.astype("float32")/255.0  
y = to_categorical(y, num_classes = 2)



#Unmodified Pre-trained Architectures
def buildmodel_pretrained_architecture():
  base_model=InceptionV3(include_top=False, weights="imagenet", input_shape=(224,224,3))
  #base_model=ResNet50(include_top=False,weights="imagenet",input_shape=(224,224,3))
  #base_model=VGG19(include_top=False,weights='imagenet', input_shape=(224,224,3))
  #base_model=VGG16(include_top=False,weights='imagenet', input_shape=(224,224,3))
  base_model.trainable=True
  model=Sequential()
  model.add(base_model)
  model.add(GlobalAveragePooling2D())  #for InceptionV3 and Resnet50
  """model.add(Flatten())   #for VGG16 and VGG19
  model.add(Dense(4096))
  model.add(Dense(4096))"""
  numberOfClass=2
  model.add(Dense(numberOfClass,activation='softmax'))
  return model

#Modified Pre-trained Architectures
def buildmodel_modified_pretrained_architecture():
  #base_model=InceptionV3(include_top=False, weights="imagenet", input_shape=(224,224,3))
  #base_model=ResNet50(include_top=False,weights="imagenet",input_shape=(224,224,3))
  base_model=VGG19(include_top=False,weights='imagenet', input_shape=(224,224,3))
  #base_model=VGG16(include_top=False,weights='imagenet', input_shape=(224,224,3))
  base_model.trainable=False
  model=Sequential()
  model.add(base_model)
  model.add(Flatten())
  model.add(Dense(32))
  model.add(Dense(64))
  model.add(Dense(128))
  model.add(Dense(256))
  numberOfClass=2
  model.add(Dense(numberOfClass,activation='softmax'))
  return model

kacfold=5
from sklearn.model_selection import KFold
kf = KFold(n_splits=kacfold, shuffle=True)
sumaccuracy=0
sumrecall=0
sumprecision=0
sumf1=0
owerlapconf_mat=0
history=[]
acc_val=[]
result=[]
aucs = []
tprs = []
fprs = []
base_fpr = np.linspace(0, 1, 101)
plt.figure(figsize=(10, 10))
fold=1
opt=keras.optimizers.Adam(lr=0.001)

k=5
if (k==0):opt =tf.optimizers.Adam(lr=0.0001)
if (k==1):opt=tf.optimizers.Adadelta(lr=0.01, rho=0.95, epsilon=None, decay=0.0)
if (k==2):opt =tf.optimizers.SGD(lr=0.01, clipnorm=1.)
if (k==3):opt =tf.optimizers.RMSprop(lr=0.001, rho=0.8, epsilon=None, decay=0.0)
if (k==4):opt=tf.optimizers.Adamax(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
if (k==5):opt=tf.optimizers.Nadam(lr=0.00002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)

for train_index, test_index in kf.split(X):
    
    x_train = X[train_index]
    y_train = y[train_index]
    x_test = X[test_index]
    y_test = y[test_index]
    model1=buildmodel_modified_pretrained_architecture()
    #model1=buildmodel_pretrained_architecture()
    model1.compile(loss="binary_crossentropy", optimizer=opt,  metrics=["accuracy"])
    history = model1.fit(x_train, y_train, batch_size=32,epochs=60, verbose=0) 
    np.save("drive/My Drive/LargeDataSetBen/modelsmodify/deneme"+str(fold)+".npy",history.history)  
    prediction=model1.predict(x_test)
    Y_pred_classes =np.argmax(prediction,axis = 1) 
    y_test_classes =np.argmax(y_test,axis = 1)
    result=metrics.classification_report(y_test_classes, Y_pred_classes, digits=4,output_dict=True)
    result2=metrics.classification_report(y_test_classes, Y_pred_classes, digits=4,output_dict=False)
    sumaccuracy=sumaccuracy+result['accuracy']
    sumrecall=sumrecall+result['weighted avg']['recall']
    sumprecision=sumprecision+result['weighted avg']['precision']
    sumf1=sumf1+result['weighted avg']['f1-score']
    conf_mat =confusion_matrix(y_test_classes, Y_pred_classes, labels=[0, 1])
    owerlapconf_mat=owerlapconf_mat+conf_mat
    print('\n',conf_mat)
    print(result2)

    probas_ = model1.predict_proba(x_test)
    fpr, tpr, thresholds = metrics.roc_curve(y_test_classes, probas_[:, 1],drop_intermediate=False,pos_label=1)
    roc_auc = metrics.auc(fpr, tpr)
    print("Auc-"+str(fold)+"=",roc_auc)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=1, alpha=0.3,label='ROC Fold- %d (AUC = %0.2f)' % (fold, roc_auc))
    tpr = np.interp(base_fpr, fpr, tpr)
    tpr[0] = 0.0
    tprs.append(tpr)

    fold=fold+1
meanaccuracy=sumaccuracy/kacfold
print("Mean Accuracy=",meanaccuracy)
print("Mean weighted avg Recall=",sumrecall/kacfold)
print("Mean weighted avg Precision=",sumprecision/kacfold)
print("Mean weighted avg F1-Score=",sumf1/kacfold)
print('\nOwerlap Confusion Matrix:\n',owerlapconf_mat)
total1=sum(sum(owerlapconf_mat))
owrecall=owerlapconf_mat[1,1]/(owerlapconf_mat[1,1]+owerlapconf_mat[1,0])
owspecifity=owerlapconf_mat[0,0]/(owerlapconf_mat[0,0]+owerlapconf_mat[0,1])
owprecisionnormal=owerlapconf_mat[0,0]/(owerlapconf_mat[0,0]+owerlapconf_mat[1,0])
owprecisioncovid=owerlapconf_mat[1,1]/(owerlapconf_mat[1,1]+owerlapconf_mat[0,1])
owf1normal=2*(owspecifity*owprecisionnormal)/(owspecifity+owprecisionnormal)
owf1covid=2*(owrecall*owprecisioncovid)/(owrecall+owprecisioncovid)
print("Owerlap Recall=",owrecall)
print("Owerlap Specificity=",owspecifity)
print("Owerlap Normal Precision=",owprecisionnormal)
print("Owerlap Covid Precision=",owprecisioncovid)
print("Owerlap Normal F1-Score=",owf1normal)
print("Owerlap Covid F1-Score=",owf1covid)

tprs = np.array(tprs)
mean_tprs = tprs.mean(axis=0)
mean_auc = metrics.auc(base_fpr, mean_tprs)
std = tprs.std(axis=0)

tprs_upper = np.minimum(mean_tprs + std, 1)
tprs_lower = mean_tprs - std
std_auc = np.std(aucs)
plt.plot(base_fpr, mean_tprs, 
          label='Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
          lw=2, alpha=.8)
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',label='Chance', alpha=.8)    
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('False Positive Rate',fontsize=18)
plt.ylabel('True Positive Rate',fontsize=18)
plt.title('Cross-Validation ROC of Model-4 ')
plt.legend(loc="lower right", prop={'size': 15})
plt.show()

In [ ]:
#%load_ext autotime
import json,codecs
import pandas as pd
import numpy as np

n=np.load("drive/My Drive/LargeDataSetBen/modelsmodify/my_historytwol2.npy",allow_pickle='TRUE').item()

#np.save("drive/My Drive/Datacovid_kemal/my_history"+str(fold)+".npy",history.history)
#n=history.history
plt.plot(n['loss'],label='train loss')
#plt.plot(n['accuracy'],label='train acc')
#plt.plot(n['val_loss'],label='validation loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend()
plt.show()

plt.plot(n['accuracy'],label='train acc')
#plt.plot(n['val_accuracy'],label='validation acc')
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.legend()
plt.show()

In [ ]:
pip install scikit-plot

In [ ]:
!pip install ipython-autotime